<a href="https://colab.research.google.com/github/s-choung/Simulation_tutorials/blob/main/handson_DFT_setup_and_bulk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Density Functional Theory in Catalyst Research

The urgent need to combat climate change has increased the focus on renewable energy sources, aiming to reduce reliance on fossil fuels and mitigate environmental harm. Efficient catalysts are essential for accelerating chemical reactions, lowering energy consumption, and improving production efficiency. Density Functional Theory (DFT) allows for the simulation of electron density around atomic nuclei by approximating the Schrödinger equation. This facilitates the prediction of catalyst properties, including active sites, adsorbate interactions, and electronic structures, driving more efficient and predictive catalyst design.

*Choung et al., Chemical Engineering Journal 494, 2024, 152757*


## The grid-based projector-augmented wave (GPAW) code
GPAW is a density-functional theory (DFT) Python code based on the projector-augmented wave (PAW) method and the atomic simulation environment (ASE).  GPAW is open-source Python package for electronic structure calculations. GPAW is based on the projector-augmented wave method and can solve the self-consistent DFT equations using three different wave-function representations, namely real-space grids, plane waves, and numerical atomic orbitals.

- **`python3-dev`**: Provides header files needed to compile Python extensions for `gpaw`'s high-performance computations.
- **`libopenblas-dev`**: Optimized library for linear algebra operations like matrix multiplication, ensuring efficient performance in `gpaw`.
- **`liblapack-dev`**: Another essential library for linear algebra tasks such as solving equations and eigenvalue problems.
- **`libfftw3-dev`**: Library for Fast Fourier Transform (FFT), used in electronic structure calculations to handle problems in reciprocal space.
- **`libxc-dev`**: Provides exchange-correlation functionals required for Density Functional Theory (DFT) calculations in `gpaw`.


In [12]:
!apt-get update
!apt-get install -y python3-dev libopenblas-dev liblapack-dev libfftw3-dev libxc-dev

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pack

In [13]:
!pip install gpaw

In [14]:
!gpaw test

 -------------------------------------------------------------------------------------------------
| python-3.10.12    /usr/bin/python3                                                              |
| gpaw-24.6.0       /usr/local/lib/python3.10/dist-packages/gpaw/                                 |
| ase-3.23.0        /usr/local/lib/python3.10/dist-packages/ase/                                  |
| numpy-1.26.4      /usr/local/lib/python3.10/dist-packages/numpy/                                |
| scipy-1.13.1      /usr/local/lib/python3.10/dist-packages/scipy/                                |
| libxc-5.1.7       yes                                                                           |
| _gpaw             /usr/local/lib/python3.10/dist-packages/_gpaw.cpython-310-x86_64-linux-gnu.so |
| MPI enabled       yes                                                                           |
| OpenMP enabled    no                                                                            |
|

## Install Pseudo potential

A setup is to the PAW method what a pseudo-potential is to the pseudo-potential method.
read more: https://gpaw.readthedocs.io/setups/setups.html

In [ ]:
!gpaw install-data ./

Available setups and pseudopotentials
  [*] https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-24.1.0.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.9.20000.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.9.11271.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.9.9672.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.8.7929.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.6.6300.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.5.3574.tar.gz

Selected gpaw-setups-24.1.0.tar.gz.  Downloading...
Extracting tarball into ./
Setups installed into /content/gpaw-setups-24.1.0.
Register this setup path in /root/.gpaw/rc.py? [y/n] 

In [ ]:
import os
import gzip
import shutil

# Define the path to the setup files
setup_path = '/content/gpaw-setups-24.1.0'

# Uncompress all .gz files in the setup directory
for filename in os.listdir(setup_path):
    if filename.endswith('.gz'):
        filepath = os.path.join(setup_path, filename)
        with gzip.open(filepath, 'rb') as f_in:
            with open(filepath[:-3], 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

# List the files after uncompressing to ensure the process completed successfully
print("Files in setup directory after uncompressing:", os.listdir(setup_path))

# Set the GPAW_SETUP_PATH environment variable
os.environ['GPAW_SETUP_PATH'] = setup_path

# Verify the environment variable
print("GPAW_SETUP_PATH:", os.environ['GPAW_SETUP_PATH'])




## Atomic Simulation Environment (ASE)
GPAW is well integrated with the Atomic Simulation Environment (ASE), providing a flexible and dynamic user interface

In [ ]:
!pip install ase

In [ ]:
# Import ASE and GPAW and run your calculation
from ase import Atoms
from gpaw import GPAW

# Define the atomic structure
d = 0.74
a = 6.0

h2 = Atoms('H2',
              positions=[(0, 0, 0),
                         (0, 0, d)],
              cell=(a, a, a))
h2.center()
# Initialize GPAW calculator without specifying the 'setups' parameter
calc = GPAW(mode='fd', nbands=2, txt='h2.txt')
h2.calc = calc

# Run the calculation
print("Forces on atoms:", h2.get_forces())
print("Energies on atoms:", h2.get_total_energy())


## Install POVRAY: visualization of atoms

The Persistence of Vision Ray Tracer, most commonly acronymed as POV-Ray, is a cross-platform ray-tracing program that generates images from a text-based scene description.

In [ ]:
!sudo apt-get install povray  ## visulization program

In [ ]:
import os
import shutil
from ase.io import write  # Import the write function from ASE
from PIL import Image
from IPython.display import display  # Import display for Jupyter Notebooks


def visual(structure, max_size=(200, 200), stretch_y=1.0, rotation='15z,-90x', png_save_path='./'):
    renderer = write('./temp.pov', structure, rotation=rotation)
    renderer.render()
    image_path = './temp.png'
    img = Image.open(image_path)

    # Calculate new size with stretch factor
    new_size = (max_size[0], int(max_size[1] * stretch_y))
    img = img.resize(new_size, Image.LANCZOS)
    display(img)

    # Move files to output directory
    files = ['./temp.ini', './temp.pov', './temp.png']
    destination = './output/'

    # Ensure destination directory exists
    os.makedirs(destination, exist_ok=True)

    for file in files:
        # Remove the file in the destination directory if it exists
        if os.path.isfile(os.path.join(destination, os.path.basename(file))):
            os.remove(os.path.join(destination, os.path.basename(file)))

        if file == './temp.png':
            new_file_name = png_save_path + f'temp.png'
        else:
            new_file_name = os.path.basename(file)
        shutil.move(file, os.path.join(destination, new_file_name))


In [ ]:
visual(bulk,(300,300))

## Bulk Calculations

In [ ]:
"""Bulk Al(fcc) test"""
from ase import Atoms
from ase.visualize import view
from gpaw import GPAW, PW

name = 'Al-fcc'
a = 4.05  # fcc lattice parameter
b = a / 2
bulk = Atoms('Al',
             cell=[[0, b, b],
                   [b, 0, b],
                   [b, b, 0]],
             pbc=True)

k = 4
calc = GPAW(mode=PW(300),       # cutoff
            kpts=(k, k, k),     # k-points
            txt=name + '.txt')  # output file

bulk.calc = calc

energy = bulk.get_potential_energy()
calc.write(name + '.gpw')
print('Energy:', energy, 'eV')

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ase import Atoms
from gpaw import GPAW, PW

# Testing k-points
k_values = [2, 4, 6, 8, 10]  # Different k-point grids to test
energies_k = []
times_k = []

a = 4.05  # fcc lattice parameter
b = a / 2

# Loop over different k-points
for k in k_values:
    # Define the bulk FCC structure
    bulk = Atoms('Al',
                 cell=[[0, b, b],
                       [b, 0, b],
                       [b, b, 0]],
                 pbc=True)

    # Set up the GPAW calculator
    calc = GPAW(mode=PW(300),  # Fixed cutoff energy
                kpts=(k, k, k),  # Varying k-points
                txt=f'Al-fcc-k{k}.txt')

    # Attach the calculator to the bulk structure
    bulk.calc = calc

    # Start timing
    start_time = time.time()

    # Calculate the potential energy
    energy = bulk.get_potential_energy()

    # End timing
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time

    # Store the energy and time
    energies_k.append(energy)
    times_k.append(elapsed_time)

    # Save the calculator state
    calc.write(f'Al-fcc-k{k}.gpw')
    del bulk.calc  # Clean up

# Plot k-points vs. energy
plt.figure()
plt.plot(k_values, energies_k, marker='o', linestyle='-')
plt.xlabel('K-points')
plt.ylabel('Energy (eV)')
plt.title('Energy vs. K-points')
plt.grid()
plt.show()

# Create a DataFrame for the time taken for different k-points
time_k_table = pd.DataFrame({'K-points': k_values, 'Time (s)': times_k})
print(time_k_table)

# Testing plane-wave cutoff energies
pw_values = np.linspace(200, 600, 5)  # Different PW values to test
energies_pw = []
times_pw = []

# Loop over different PW cutoff energies
for pw in pw_values:
    # Define the bulk FCC structure
    bulk = Atoms('Al',
                 cell=[[0, b, b],
                       [b, 0, b],
                       [b, b, 0]],
                 pbc=True)

    # Set up the GPAW calculator
    calc = GPAW(mode=PW(pw),  # Varying cutoff energy
                kpts=(4, 4, 4),  # Fixed k-points
                txt=f'Al-fcc-pw{pw:.0f}.txt')

    # Attach the calculator to the bulk structure
    bulk.calc = calc

    # Start timing
    start_time = time.time()

    # Calculate the potential energy
    energy = bulk.get_potential_energy()

    # End timing
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time

    # Store the energy and time
    energies_pw.append(energy)
    times_pw.append(elapsed_time)

    # Save the calculator state
    calc.write(f'Al-fcc-pw{pw:.0f}.gpw')
    del bulk.calc  # Clean up

# Plot PW values vs. energy
plt.figure()
plt.plot(pw_values, energies_pw, marker='o', linestyle='-')
plt.xlabel('PW Cutoff (eV)')
plt.ylabel('Energy (eV)')
plt.title('Energy vs. PW Cutoff')
plt.grid()
plt.show()

# Create a DataFrame for the time taken for different PW values
time_pw_table = pd.DataFrame({'PW Cutoff (eV)': pw_values, 'Time (s)': times_pw})
print(time_pw_table)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ase import Atoms
from gpaw import GPAW, PW

# Set up values for lattice parameter 'a'
a_values = np.linspace(3.8, 4.2, 10)  # Adjust the range and number of points as needed
energies = []
volumes = []

# Loop over each lattice parameter to calculate energy
for a in a_values:
    b = a / 2
    # Define the bulk structure
    bulk = Atoms('Al',
                 cell=[[0, b, b],
                       [b, 0, b],
                       [b, b, 0]],
                 pbc=True)

    # Set up the calculator
    k = 4
    calc = GPAW(mode=PW(300),       # Plane-wave cutoff
                kpts=(k, k, k),     # k-points grid
                txt=f'Al-fcc-a{a:.2f}.txt')  # Output file with varying name

    # Attach the calculator to the bulk structure
    bulk.calc = calc

    # Calculate the potential energy
    energy = bulk.get_potential_energy()
    volume = bulk.get_volume()

    # Store the energy and volume
    energies.append(energy)
    volumes.append(volume)

    # Clean up the calculator
    calc.write(f'Al-fcc-a{a:.2f}.gpw')
    del bulk.calc  # Free memory

# Plot EOS: Energy vs. Volume
plt.figure()
plt.plot(volumes, energies, marker='o', linestyle='-')
plt.xlabel('Volume (Å³)')
plt.ylabel('Energy (eV)')
plt.title('Equation of State for Al (FCC)')
plt.grid()
plt.show()


slab

In [ ]:
import os
import shutil
from ase.io import write  # Import the write function from ASE
from PIL import Image
from IPython.display import display  # Import display for Jupyter Notebooks


def visual(structure, max_size=(100, 100), stretch_y=1.0, rotation='15z,-90x', png_save_path='./'):
    renderer = write('./temp.pov', structure, rotation=rotation)
    renderer.render()
    image_path = './temp.png'

    # Calculate new size with stretch factor
    new_size = (max_size[0], int(max_size[1] * stretch_y))
    img = img.resize(new_size, Image.LANCZOS)
    display(img)
    # Move files to output directory
    files = ['./temp.ini', './temp.pov', './temp.png']
    destination = './output/'

    # Ensure destination directory exists
    os.makedirs(destination, exist_ok=True)

    for file in files:
        # Remove the file in the destination directory if it exists
        if os.path.isfile(os.path.join(destination, os.path.basename(file))):
            os.remove(os.path.join(destination, os.path.basename(file)))

        if file == './temp.png':
            new_file_name = png_save_path + f'temp.png'
        else:
            new_file_name = os.path.basename(file)
        shutil.move(file, os.path.join(destination, new_file_name))


In [ ]:
visual(bulk,(300,300))
repeat=(4,4,4)
visual(bulk*repeat,(300,300))

In [ ]:
import time  # Import the time module
from ase.build import fcc100
from gpaw import GPAW

# Initialize the parameters
k = 1
N = 4
size = 2
# Create the slab
fcc = fcc100('Al', (size, size, N), a=4, vacuum=7.5)
fcc.center(axis=2)

# Set up the GPAW calculator
calc = GPAW(mode='fd',
            nbands=N * 3 * size**2,
            kpts=(k, k, 1),
            h=0.25,
            txt='slab-%d.txt' % N)

# Attach the calculator to the slab
fcc.calc = calc

# Start timing
start_time = time.time()

# Run the calculation to get the potential energy
e = fcc.get_potential_energy()

# End timing
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Save the calculator state
calc.write('slab-%d.gpw' % N)

# Print the energy and the elapsed time
print(f'Potential Energy: {e} eV')
print(f'Time taken: {elapsed_time:.2f} seconds')
